In [ ]:
pip install torch torchvision numpy pillow


In [ ]:
import torch
import torch.nn as nn

class ECGGenerator3D(nn.Module):
    def __init__(self, input_dim, output_channels, base_dim):
        super(ECGGenerator3D, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, base_dim * 4 * 4 * 4),
            nn.ReLU(inplace=True),
            nn.Unflatten(1, (base_dim, 4, 4, 4)),
            nn.ConvTranspose3d(base_dim, base_dim // 2, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm3d(base_dim // 2),
            nn.ReLU(inplace=True),
            nn.ConvTranspose3d(base_dim // 2, base_dim // 4, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm3d(base_dim // 4),
            nn.ReLU(inplace=True),
            nn.ConvTranspose3d(base_dim // 4, output_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, z):
        return self.model(z)


In [5]:
import torch
import torch.nn as nn

class ECGDiscriminator3D(nn.Module):
    def __init__(self, input_channels, base_dim):
        super(ECGDiscriminator3D, self).__init__()
        self.model = nn.Sequential(
            nn.Conv3d(input_channels, base_dim, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.BatchNorm3d(base_dim),
            nn.Conv3d(base_dim, base_dim * 2, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.BatchNorm3d(base_dim * 2),
            nn.Conv3d(base_dim * 2, base_dim * 4, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.BatchNorm3d(base_dim * 4),
            nn.Flatten(),
            nn.Linear(base_dim * 4 * 16 * 16 * 16, 1),  # Adjust size based on input
            nn.Sigmoid()  # Output a probability (real/fake)
        )

    def forward(self, x):
        return self.model(x)



In [9]:
import os
import numpy as np
from PIL import Image

def load_and_preprocess_data(folder_path):
    # List to store preprocessed ECG images
    ecg_images = []
    
    # Walk through the folder and process images
    for root, _, files in os.walk(folder_path):
        for file_name in files:
            if file_name.endswith('.png'):
                try:
                    # Get the full path of the image
                    file_path = os.path.join(root, file_name)
                    
                    # Open the image, convert to grayscale (L mode), and resize to 128x128
                    image = Image.open(file_path).convert('L')
                    image = image.resize((128, 128))  # Resize to save memory
                    
                    # Convert to NumPy array, normalize to [0, 1]
                    image_array = np.array(image, dtype=np.float32) / 255.0
                    
                    # Add to list
                    ecg_images.append(image_array)
                except Exception as e:
                    print(f"Error processing image {file_name}: {e}")

    # Check if no images were loaded
    if len(ecg_images) == 0:
        raise FileNotFoundError("No valid ECG image files found in the specified folder.")

    # Convert list to NumPy array and return
    return np.stack(ecg_images)  # Stack the images into a single array

# Example usage
folder_path = "C:\\Users\\MTechS_h\\Desktop\\Mini Project\\ecg_data\\ECG_Image_data"
try:
    ecg_data = load_and_preprocess_data(folder_path)
    print(f"Loaded ECG data with shape: {ecg_data.shape}")
except FileNotFoundError as e:
    print(e)



Loaded ECG data with shape: (123998, 128, 128)


In [7]:
criterion = nn.BCELoss()


In [16]:
import torch

# Define the device to use (either CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize models and move them to the correct device
generator = ECGGenerator3D(input_dim=100, output_channels=1, base_dim=64).to(device)
discriminator = ECGDiscriminator3D(input_channels=1, base_dim=64).to(device)



In [18]:
# Initialize models
generator = ECGGenerator3D(input_dim=100, output_channels=1, base_dim=64).to(device)
discriminator = ECGDiscriminator3D(input_channels=1, base_dim=64).to(device)

# Load the ECG data
ecg_data = load_and_preprocess_data("C:\Users\MTechS_h\Desktop\Mini Project\ecg_data\ECG_Image_data")
ecg_data = torch.tensor(ecg_data).unsqueeze(1)  # Add channel dimension
data_loader = torch.utils.data.DataLoader(ecg_data, batch_size=64, shuffle=True)

# Training loop
for epoch in range(num_epochs):
    for real_data in data_loader:
        real_data = real_data.to(device)

        # 1. Train Discriminator
        optimizer_D.zero_grad()

        # Discriminator loss on real data
        real_output = discriminator(real_data)
        real_loss = criterion(real_output, real_labels)

        # Generate fake data
        z = torch.randn(batch_size, 100).to(device)
        fake_data = generator(z)

        # Discriminator loss on fake data
        fake_output = discriminator(fake_data.detach())  # Detach to not update Generator
        fake_loss = criterion(fake_output, fake_labels)

        # Total Discriminator loss
        d_loss = real_loss + fake_loss
        d_loss.backward()
        optimizer_D.step()

        # 2. Train Generator
        optimizer_G.zero_grad()

        # Generator tries to fool the Discriminator
        fake_output = discriminator(fake_data)
        g_loss = criterion(fake_output, real_labels)  # Generator wants Discriminator to think it's real
        g_loss.backward()
        optimizer_G.step()

    print(f"Epoch [{epoch}/{num_epochs}], D Loss: {d_loss.item()}, G Loss: {g_loss.item()}")


FileNotFoundError: No valid ECG image files found in the specified folder.

In [17]:
z = torch.randn(1, 100).to(device)
generated_ecg = generator(z)
